# User Defined Equations

Examples:

- find value of evaporation temperature to achieve specific COP
- a custom superheat specification
- set the minimal pinch across a number of heat exchangers

## General info

- Formulate an equation $0 = ...$ in a custom function and return residual
- Specify which variables the equation depends on in a second function
- Pass all other relevant information to an instance of UserDefinedEquation
  - connections available through ude.conns
  - components available through ude.comps
  - function to calculate the partial derivatives explicitly instead of doing
    it via dependents
  - other parameters (dictionary) available through ude.params
- For more, readthedocs: https://tespy.readthedocs.io/en/main/modules/ude.html

## Custom superheat

- heat pump
- fixed efficiency of compressor
- fixed subcooling
- fixed evaporation and condensation temperature level
- target amount of heat production by condenser
- a bit of superheating with a pressure loss
- goal: specify superheat with respect to evaporator pressure outlet


In [1]:
from tespy.components import SimpleHeatExchanger, CycleCloser, Valve, Compressor
from tespy.connections import Connection
from tespy.networks import Network

In [2]:
nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar",
    power="kW",
    heat="kW"
)

In [3]:
evaporator = SimpleHeatExchanger("evaporator")
superheater = SimpleHeatExchanger("superheater")
compressor = Compressor("compressor")
condenser = SimpleHeatExchanger("condenser")
valve = Valve("valve")
cc = CycleCloser("cc")

c1 = Connection(evaporator, "out1", superheater, "in1", label="c1")
c1b = Connection(superheater, "out1", compressor, "in1", label="c1b")
c2 = Connection(compressor, "out1", condenser, "in1", label="c2")
c3 = Connection(condenser, "out1", valve, "in1", label="c3")
c4 = Connection(valve, "out1", cc, "in1", label="c4")
c5 = Connection(cc, "out1", evaporator, "in1", label="c5")

nw.add_conns(c1, c1b, c2, c3, c4, c5)

In [4]:
c1.set_attr(fluid={"R600": 1}, T_dew=10, td_dew=2)
c1b.set_attr(td_dew=12)
c3.set_attr(T_bubble=60, td_bubble=5)

evaporator.set_attr(dp=0)
superheater.set_attr(dp=1)
condenser.set_attr(dp=0, Q=-100)
compressor.set_attr(eta_s=0.8)

nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 3.26e+05   | 5 %        | 1.01e+00   | 0.00e+00   | 2.30e+04   | 0.00e+00   | 0.00e+00   
 2     | 2.33e+04   | 18 %       | 6.25e-02   | 0.00e+00   | 9.09e-11   | 0.00e+00   | 0.00e+00   
 3     | 2.02e-09   | 100 %      | 5.36e-15   | 0.00e+00   | 9.09e-11   | 0.00e+00   | 0.00e+00   
 4     | 7.42e-11   | 100 %      | 2.65e-17   | 0.00e+00   | 9.09e-11   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: 1548.28


In [5]:
# superheating is 12 K, but temperature is actually lower than before due to
# pressure decrease
c1b.T.val

-6.452407301245785

In [6]:
from tespy.tools import UserDefinedEquation
from tespy.tools.fluid_properties.functions import T_dew_p


def superheat_ude(ude):
    c1, c1b = ude.conns
    return c1b.calc_T() - T_dew_p(c1.p.val_SI, c1.fluid_data) - ude.params["superheat"]

def superheat_ude_dependents(ude):
    c1, c1b = ude.conns
    return [c1.p, c1b.p, c1b.h]


shude = UserDefinedEquation(
    "shude",
    superheat_ude,
    superheat_ude_dependents,
    conns=[c1, c1b],
    params={"superheat": 10}
)
c1b.set_attr(td_dew=None)
nw.add_ude(shude)
nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 2.65e+01   | 50 %       | 4.05e-02   | 0.00e+00   | 7.02e+04   | 0.00e+00   | 0.00e+00   
 2     | 2.35e+03   | 29 %       | 4.76e-03   | 0.00e+00   | 1.69e+03   | 0.00e+00   | 0.00e+00   
 3     | 4.98e+00   | 58 %       | 1.21e-05   | 0.00e+00   | 1.67e+00   | 0.00e+00   | 0.00e+00   
 4     | 1.31e-05   | 100 %      | 3.02e-11   | 0.00e+00   | 1.22e-06   | 0.00e+00   | 0.00e+00   
 5     | 7.28e-11   | 100 %      | 3.35e-16   | 0.00e+00   | 1.05e-09   | 0.00e+00   | 0.00e+00   
Total iterations: 5, Calculation time: 0.01 s, Iterations per second: 746.90


In [7]:
c1b.T.val

20.000000000000057